# Parsing of the bokeh layout playing

In [1]:
import pyparsing 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import *
from bokeh.models import HoverTool
output_notebook()
import random
from bokeh.palettes import Spectral6 as palette
import itertools
colors = itertools.cycle(palette)

Loading BokehJS ...

# generate array of random figures

In [36]:
npoints=50
nfigures=10
x = list(range(npoints))
yList=[]
figList=[]
tooltips=[("x","(@x)"),  ("y","@y")]
tools = 'pan,box_zoom, wheel_zoom,box_select,lasso_select,reset'
tools = [HoverTool(tooltips=tooltips), tools]
for i in range(nfigures):
    y=random.sample(xrange(50), npoints)
    if (i>0): 
        for idx, val in enumerate(y):
            y[idx]+=yList[i-1][idx]
    y.sort()
    yList.append(y)
    f = figure(width=250, plot_height=250,tools=tools)
    f.circle(x, y, size=10, color=next(colors), alpha=0.5)
    figList.append(f)
f=figList    

In [38]:
def processRow(row, figList,layoutList,optionsMother):
    print("Raw",row)
    array=[]
    layoutList.append(array)
    option=processLayoutOption(row)
    print("Option",option)
    for idx,y in enumerate(row):
        if not y.isdigit(): continue 
        fig=figList[int(y)]
        array.append(fig)
        if ('commonY' in option):
            if type(option["commonY"])==str: 
                fig.y_range=array[0].y_range
            else:
                try: fig.y_range=figList[int(option["commonY"])].y_range
                except ValueError: continue
        if ('commonX' in option):
            if type(option["commonX"])==str: 
                fig.x_range=array[0].x_range
            else:
                try: fig.x_range=figList[int(option["commonX"])].x_range
                except ValueError,e : 
                    logger.error('Failed: to processs option '+ option["commonX"])

            
        if (idx>0) & ('y_visible' in option): fig.yaxis.visible=bool(option["y_visible"])
        if ('x_visible' in option):     fig.xaxis.visible=bool(option["x_visible"])
    nCols=len(array)
    for fig in array:
        if ('plot_width' in option):
            fig.plot_width=int(option["plot_width"])/nCols
        else:
            if ('plot_width' in optionsMother): 
                fig.plot_width=int(optionsMother["plot_width"])/nCols  
        if ('plot_height' in option): 
            fig.plot_height=int(option["plot_height"])
        else: 
            if ('plot_height' in optionsMother): 
                fig.plot_height=int(optionsMother["plot_height"])
                
    
            
           
def processLayoutOption(layout):   # https://stackoverflow.com/questions/9305387/string-of-kwargs-to-kwargs
    options={}
    for x in layout: 
        if not(type(x) == str): continue
        if ("=" in str(x)):                  ## one of the way to see if it's list
            try:
                k, v=x.split("=")
            except ValueError: continue   
            options[k]=v
            if v.isdigit():
                options[k]=int(v)
            else:
                try:
                    options[k]=float(v)
                except ValueError:
                    options[k]=v 
    return options

def processLayout(layout, figList):
    thecontent = pyparsing.Word(pyparsing.alphanums+".+-=_")  | pyparsing.Suppress(',')
    parens     = pyparsing.nestedExpr( '(', ')', content=thecontent)
    res = parens.parseString(layout)[0]
    layoutList=[]
    optionList=[]
    #print(res.asList())
    options=processLayoutOption(res)
    print(options)
    for x in res:
        if type(x) != str:  
           processRow(x,figList,layoutList,options)
    # remove plot_withth and plot_height form options - set in individual figures    
    if "plot_width" in options:    del options["plot_width"]
    if "plot_height" in options:    del options["plot_height"]    
    return res.asList(), layoutList, options

In [44]:
layout="((0,2,3,commonY=1,commonX=1,x_visible=1,y_visible=0), (1,commonY=1,commonX=1,plot_height=80, x_visible=0),"
layout+="(4,commonY=1,commonX=1,plot_height=80), plot_width=900, plot_height=200)"
x,layoutList,options=processLayout(layout,figList)
show(gridplot(layoutList,**options))

{'plot_height': 200, 'plot_width': 900}
('Raw', (['0', '2', '3', 'commonY=1', 'commonX=1', 'x_visible=1', 'y_visible=0'], {}))
('Option', {'x_visible': 1, 'y_visible': 0, 'commonX': 1, 'commonY': 1})
('Raw', (['1', 'commonY=1', 'commonX=1', 'plot_height=80', 'x_visible=0'], {}))
('Option', {'plot_height': 80, 'x_visible': 0, 'commonX': 1, 'commonY': 1})
('Raw', (['4', 'commonY=1', 'commonX=1', 'plot_height=80'], {}))
('Option', {'plot_height': 80, 'commonX': 1, 'commonY': 1})
